In [30]:
import numpy as np
import json, copy, os, glob
import scipy.integrate as spi
import astropy.units as u
from astropy.coordinates import SkyCoord
import astropy.cosmology.units as cu
import pandas as pd
from enterprise import constants as const

Priors files will contain the following:
- RA (arcseconds)
- DEC (arcseconds)
- log10_dist (Mpc)
- log10_freq (Hz)
- log10_Mc ($M_\odot$)
- log10_Mc_sigma ($M_\odot$)

### Luminosity distance

Xin+2021 & SDSS catalog reports redshifts instead of distances. In cases where I don't have distances already, The formula to convert is
\begin{align*}
    D_L &= \frac{c}{H_0}(1+z)\int_0^z\frac{dz}{\sqrt{\Omega_{m,0}(1+z)^3 + \Omega_{\Lambda,0}}}.
\end{align*}
We will use $H_0 = 69.33$ km/s/Mpc, $\Omega_{m,0} = 0.2880$, $\Omega_{\Lambda,0} = 0.7120$, from Himshaw+2013/WMAP/cosmopy for consistency with the 15yr CW search paper, and $c = 299792.458$ km/s.

### Chirp mass

Convert total mass to chirp mass as
\begin{align*}
    \mathcal{M}^{5/3} &= \frac{q}{(1+q)^2}M^{5/3},
\end{align*}
or,
\begin{align*}
    \log_{10}\mathcal{M} &= \log_{10}M + \frac{3}{5}\log_{10}q - \frac{6}{5}\log_{10}\left(1+q\right).
\end{align*}
Note table 1 in Xin et al 2021 is all total masses.

### Observed GW frequency

Recall the GW frequency is twice the orbital frequency, thus
\begin{align*}
    f_{\rm{GW}} = \frac{2}{T_0}
\end{align*}
where $T_0$ is the observed orbital period. If converting from the rest-frame orbital period $T_r$, then we also have the redshift dependent factor
\begin{align*}
    f_{\rm{GW}} = \frac{2}{(1 + z)T_r}.
\end{align*}

In [31]:
def z_to_dist_L(z, H0=69.33, Omega_L=0.7120, Omega_m=0.2880, c=299792.458):
    '''
        For consistency with the 15yr paper, using the cosmology parameters
        from Hinshaw+2013 (WMAP)
    '''
    def integrand(z, Omega_m, Omega_L):
        return (Omega_m*(1+z)**3 + Omega_L)**(-1/2)
    DL = c*(1+z)*spi.quad(integrand, 0, z, args=(Omega_m, Omega_L))[0]/H0
    DL_sigma = c*(1+z)*spi.quad(integrand, 0, z, args=(Omega_m, Omega_L))[1]/H0
    return DL, DL_sigma

def log10_Mtot_to_Mc(log10_Mtot, q=1):
    return log10_Mtot + 3/5*np.log10(q) - 6/5*np.log10(1+q)

def period_to_log10_fgw(T, T_units='yr', frame='obs', z=None):
    '''
    Input T in units of T_units
    '''
    if T_units == 'yr':
        units = const.yr
    elif T_units == 'day':
        units = const.day
    if frame == 'obs':
        log10_fgw = np.log10(2) - np.log10(T*units)
    elif frame == 'rest':
        log10_fgw = np.log10(2) - np.log10(T*units) - np.log10(1 + z)
    return log10_fgw

# Make pandas dataframe with all targets info

We'll format these as a pandas dataframe.

dictionaries as dictionaries of lists with [value, citation] where citation indicates where the information comes from, and the keys are the targets. If the target is something like a chirp mass, then the list is formatted [value, uncertainty, citation].

In [6]:
project_path = '/vast/palmer/home.grace/bbl29/targeted_cws_ng15'

In [32]:
# Define the columns for the DataFrame
columns = ['Target', 'Type', 'RA', 'DEC', 'RA/DEC citation',
           'z', 'log10_dist', 'z citation', # dist in Mpc
           'log10_Mc', 'log10_Mc_sigma', 'q', 'Mass UL', 'mass citation', # Mc in Msun
           'log10_freq', 'freq citation', 'extra metadata'] # freq in Hz

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)

# Example: Adding data for a few objects
data = [
    # 3C66B - Done!
    {'Target': '3C66B', 'Type': 'Radio', 
     'RA': '02h23m11.5s', 'DEC': '+42d59m30s', 'RA/DEC citation': 'Fey+2004, Xin+2021',
     'Redshift': 0.0215, 'log10_dist': np.log10(z_to_dist_L(0.0215)[0]), 'z citation': 'Matthews+1964',
     'log10_Mc': np.log10(7.9e8), 'log10_Mc_sigma': '8.53, 9.07', 'q': 0.583,
     'Mass UL': 1.34e9, 'mass citation': 'Iguchi+2010, Arzoumanian+2023', #Xin+2021
     'log10_freq': np.log10(6.04e-08), 'freq citation': 'Sudou+2003, Xin+2021',
     'extra metadata': None},
    # Here are Xin+2021 values for chirp mass
    # 'log10_Mc': log10_Mtot_to_Mc(9.08), 'log10_Mc_sigma': '7.9e8', 'q': 1,
    # CSO_J1735+5049 - Done! Don't include this in the paper though
    {'Target': 'CSO_J1735+5049', 'Type': 'CSO', 
     'RA': '17h35m49.01s', 'DEC': '+50d49m11.57s', 'RA/DEC citation': 'Kiehlmann+2024',
     'Redshift': 0.835, 'log10_dist': np.log10(z_to_dist_L(0.835)[0]), 'z citation': 'Kiehlmann+2024',
     'log10_Mc': None, 'log10_Mc_sigma': None, 'q': None, 'Mass UL': None, 'mass citation': None,
     'log10_freq': np.log10(2/(2417*u.day.to(u.s))), 'freq citation': 'From Chiara', 'extra metadata': None},
    # HS_0926+3608 - Done!
    {'Target': 'HS_0926+3608', 'Type': 'AGN', 
     'RA': '09h29m52.1s', 'DEC': '+35d54m49.6s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 2.150, 'log10_dist': np.log10(z_to_dist_L(2.150)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.95), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.48e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # HS_1630+2355 - Done!
    {'Target': 'HS_1630+2355', 'Type': 'AGN', 
     'RA': '16h33m02.7s', 'DEC': '+23d49m28.8s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 0.82101, 'log10_dist': np.log10(z_to_dist_L(0.82101)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.86), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': 1.28e10, 'mass citation': 'Graham+2015, Xin+2021, Arzoumanian+2023',
     'log10_freq': np.log10(11.3e-9), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # NGC_3115 - Done!
    {'Target': 'NGC_3115', 'Type': 'Radio', 
     'RA': '10h05m13.927s', 'DEC': '-07d43m06.983s', 'RA/DEC citation': 'SIMBAD',
     'Redshift': 0.002222, 'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), 'z citation': 'SIMBAD',
     'log10_Mc': log10_Mtot_to_Mc(np.log10(2e9)), 'log10_Mc_sigma': None, 'q': 1,
     'Mass UL': None, 'mass citation': 'Wong+2021',
     'log10_freq': None, 'freq citation': None,
     'extra metadata': None},
    # NGC_3115_49nHz - Done!
    {'Target': 'NGC_3115_49nHz', 'Type': 'Radio', 
     'RA': '10h05m13.927s', 'DEC': '-07d43m06.983s', 'RA/DEC citation': 'SIMBAD',
     'Redshift': 0.002222, 'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), 'z citation': 'SIMBAD',
     'log10_Mc': log10_Mtot_to_Mc(np.log10(2e9)), 'log10_Mc_sigma': None, 'q': 1,
     'Mass UL': None, 'mass citation': 'Wong+2021',
     'log10_freq': -7.31, 'freq citation': None,
     'extra metadata': None},
    # OJ287 - Done!
    {'Target': 'OJ287', 'Type': 'Blazar', 
     'RA': '08h54m48.9s', 'DEC': '+20d06m31s', 'RA/DEC citation': 'Xin+2021',
     'Redshift': 0.306, 'log10_dist': np.log10(z_to_dist_L(0.306)[0]), 'z citation': 'Dey+2021',
     'log10_Mc': log10_Mtot_to_Mc(10.26, q=0.008), 'log10_Mc_sigma': None, 'q': 0.008,
     'mass citation': 'Laine+2020, Xin+2021',
     'log10_freq': np.log10(5.82e-09), 'freq citation': 'Laine+2020, Xin+2021',
     'extra metadata': 'Komossa+2024 reports much lower mass measurement'},
    # PKS_2131-021 - Done!
    {'Target': 'PKS_2131-021', 'Type': 'Blazar', 
     'RA': '21h34m10.309s', 'DEC': '-01d53m17.238s', 'RA/DEC citation': 'Charlot+2020 (SIMBAD)',
     'Redshift': 1.285, 'log10_dist': np.log10(z_to_dist_L(1.285)[0]), 'z citation': "O'Neill+2022",
     'log10_Mc': None, 'log10_Mc_sigma': None, 'q': None,
     'Mass UL': (1+1.285)*4e9, 'mass citation': "Aggarwal+2019, O'Neill+2022",
     'log10_freq': period_to_log10_fgw(2.082, frame='rest', z=1.285),
     'freq citation': "O'Neill+2022, Kiehlmann+2024",
     'extra metadata': None},
    # PKS_J0805-0111 - Done!
    {'Target': 'PKS_J0805-0111', 'Type': 'Blazar', 
     'RA': '08h05m12.888s', 'DEC': '-01d11m13.795s', 'RA/DEC citation': 'Douglas+1996 (SIMBAD)',
     'Redshift': 1.388, 'log10_dist': np.log10(z_to_dist_L(1.388)[0]), 'z citation': 'de la Parra+2024',
     'log10_Mc': None, 'log10_Mc_sigma': None, 'q': None,
     'Mass UL': 5e9*(1+1.388), 'mass citation': 'Agazie+2023, de la Parra+2024',
     'log10_freq': period_to_log10_fgw(1.422, frame='rest', z=1.388), 'freq citation': 'de la Parra+2024',
     'extra metadata': None},
    # SDSS_J092911.35+2037 - Done!
    {'Target': 'SDSS_J092911.35+2037', 'Type': 'AGN', 
     'RA': '09h29m11.3s', 'DEC': '+20d37m09.2s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 1.8491, 'log10_dist': np.log10(z_to_dist_L(1.8491)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.92), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.30e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J114857.33+1600 - Done!
    {'Target': 'SDSS_J114857.33+1600', 'Type': 'AGN', 
     'RA': '11h48m57.4s', 'DEC': '+16d00m22.7s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 1.224, 'log10_dist': np.log10(z_to_dist_L(1.224)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.9), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.25e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J131706.19+2714 - Done!
    {'Target': 'SDSS_J131706.19+2714', 'Type': 'AGN', 
     'RA': '13h17m06.2s', 'DEC': '+27d14m16.7s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 2.6681, 'log10_dist': np.log10(z_to_dist_L(2.6681)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.92), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.39e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J133516.17+1833 - Done!
    {'Target': 'SDSS_J133516.17+1833', 'Type': 'AGN', 
     'RA': '13h35m16.1s', 'DEC': '+18d33m41.8s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 1.19170, 'log10_dist': np.log10(z_to_dist_L(1.19170)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.76), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.34e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J134855.27-0321 - Done!
    {'Target': 'SDSS_J134855.27-0321', 'Type': 'AGN', 
     'RA': '13h48m55.3s', 'DEC': '-3d21m41.4s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 2.0986, 'log10_dist': np.log10(z_to_dist_L(2.0986)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.89), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.62e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J140704.43+2735 - Done!
    {'Target': 'SDSS_J140704.43+2735', 'Type': 'AGN', 
     'RA': '14h07m04.5s', 'DEC': '+27d35m56.3s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 2.2162, 'log10_dist': np.log10(z_to_dist_L(2.2162)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.94), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.48e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J160730.33+1449 - Done!
    {'Target': 'SDSS_J160730.33+1449', 'Type': 'AGN', 
     'RA': '16h07m30.3s', 'DEC': '+14d49m04.2s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 1.8013, 'log10_dist': np.log10(z_to_dist_L(1.8013)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.82), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.34e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SDSS_J164452.71+4307 - Done!
    {'Target': 'SDSS_J164452.71+4307', 'Type': 'AGN', 
     'RA': '16h44m52.7s', 'DEC': '+43d07m52.9s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 1.712983, 'log10_dist': np.log10(z_to_dist_L(1.712983)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(10.15), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.16e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SNU_J13120+0641 - Done!
    {'Target': 'SNU_J13120+0641', 'Type': 'AGN', 
     'RA': '13h12m04.7s', 'DEC': '+06d41m07.6s', 'RA/DEC citation': 'Graham+2015, Xin+2021',
     'Redshift': 0.24182, 'log10_dist': np.log10(z_to_dist_L(0.24182)[0]), 'z citation': 'Graham+2015',
     'log10_Mc': log10_Mtot_to_Mc(9.14), 'log10_Mc_sigma': 0.3, 'q': 1,
     'Mass UL': None, 'mass citation': 'Graham+2015, Xin+2021',
     'log10_freq': np.log10(1.55e-08), 'freq citation': 'Graham+2015, Xin+2021',
     'extra metadata': None},
    # SNU_J13120+0641 - Done!
    {'Target': 'MDC', 'Type': 'Mock', 
     'RA': '191d00m00s', 'DEC': '53d24m00s', 'RA/DEC citation': 'Mock',
     'Redshift': None, 'log10_dist': np.log10(75.4), 'z citation': 'Mock',
     'log10_Mc': None, 'log10_Mc_sigma': None, 'q': 1,
     'Mass UL': None, 'mass citation': None,
     'log10_freq': np.log10(3.7e-09), 'freq citation': 'Mock',
     'extra metadata': None},
]

more_targets_path = '/gpfs/gibbs/project/mingarelli/frh7/targeted_searches/data/priors/'
for path in glob.glob(f'{more_targets_path}/*.json'):
    target = path.split('/')[-1][:-5].replace('_priors copy','')
    with open(path,'r') as f:
        info = json.load(f)
    info['Target'] = target
    if np.any([d['Target'] == target for d in data]):
        print(f'skipping duplicate entry for {target}')
    else:
        data.append(info)

more_targets_path = '/gpfs/gibbs/project/mingarelli/rjs235/FR_Targeted_Searches/target_priors/'
for path in glob.glob(f'{more_targets_path}/*.json'):
    target = path.split('/')[-1][4:-5].replace('_priors copy','')
    with open(path,'r') as f:
        info = json.load(f)
    info['Target'] = target
    if np.any([d['Target'] == target for d in data]):
        print(f'skipping duplicate entry for {target}')
    else:
        data.append(info)

# Add the data to the DataFrame
#for obj in data:
#    df = df.append(obj, ignore_index=True)
df = pd.DataFrame(data)
#df = df.set_index('Target')

# save to file
df.to_csv(f'{project_path}/priors/all_targets_info.csv', index=False)

# Show the DataFrame
df

skipping duplicate entry for RXS_J10304+5516
skipping duplicate entry for SDSS_J110554.78+322953.7
skipping duplicate entry for CT_638
skipping duplicate entry for SDSS_J123821.84+030024.2
skipping duplicate entry for HS_0926+3608
skipping duplicate entry for 2MASS_J04352649-1643460
skipping duplicate entry for HS_2219+1944
skipping duplicate entry for MCG_5-40-026
skipping duplicate entry for SNU_J13120+0641
skipping duplicate entry for HS_1630+2355
skipping duplicate entry for SDSS_J133631.45+175613.8
skipping duplicate entry for BZQJ1305-1033
skipping duplicate entry for SDSS_J135225.80+132853.2


,Target,Type,RA,DEC,RA/DEC citation,Redshift,log10_dist,z citation,log10_Mc,log10_Mc_sigma,q,Mass UL,mass citation,log10_freq,freq citation,extra metadata
0,3C66B,Radio,02h23m11.5s,+42d59m30s,"Fey+2004, Xin+2021",0.021500,1.975545,Matthews+1964,8.897627,"8.53, 9.07",0.583,1.340000e+09,"Iguchi+2010, Arzoumanian+2023",-7.218963,"Sudou+2003, Xin+2021",None
1,CSO_J1735+5049,CSO,17h35m49.01s,+50d49m11.57s,Kiehlmann+2024,0.835000,3.730328,Kiehlmann+2024,NaN,None,NaN,NaN,None,-8.018760,From Chiara,None
2,HS_0926+3608,AGN,09h29m52.1s,+35d54m49.6s,"Graham+2015, Xin+2021",2.150000,4.238682,Graham+2015,9.588764,0.3,1.000,NaN,"Graham+2015, Xin+2021",-7.829738,"Graham+2015, Xin+2021",None
3,HS_1630+2355,AGN,16h33m02.7s,+23d49m28.8s,"Graham+2015, Xin+2021",0.821010,3.721260,Graham+2015,9.498764,0.3,1.000,1.280000e+10,"Graham+2015, Xin+2021, Arzoumanian+2023",-7.946922,"Graham+2015, Xin+2021",None
4,NGC_3115,Radio,10h05m13.927s,-07d43m06.983s,SIMBAD,0.002222,0.983399,SIMBAD,8.939794,None,1.000,NaN,Wong+2021,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,HS_0423+0658,NaN,04h26m30.20s,+07d05m30.3s,NaN,NaN,2.848004,NaN,NaN,None,NaN,NaN,NaN,-7.986893,NaN,NaN
123,3C_298.0,NaN,14h19m08.20s,+06d28m35.1s,NaN,NaN,3.634941,NaN,NaN,None,NaN,NaN,NaN,-8.228770,NaN,NaN
124,SDSS_J134820.42+194831.5,NaN,13h48m20.40s,+19d48m31.9s,NaN,NaN,3.344942,NaN,NaN,None,NaN,NaN,NaN,-8.078903,NaN,NaN
125,FBQS_J081740.1+232731,NaN,08h17m40.20s,+23d27m32.0s,NaN,NaN,3.487294,NaN,NaN,None,NaN,NaN,NaN,-8.012061,NaN,NaN


# Old per-target dictionaries

### 1: 3C66B

In [352]:
np.log10(z_to_dist_L(0.0215)[0])

1.9755447815530847

In [289]:
# informative mass prior
source_name = '3C66B'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '02h23m11.5s', # From Xin+2021
               'DEC': '+42d59m30s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(0.0215)[0]), # redshift from Matthews et al 1964
               #'log10_dist': np.log10(93.6), # From London/Nikita Slides
               'log10_freq': np.log10(6.04e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.08), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [290]:
# uninformative mass prior (log-uniform)
source_name = '3C66B_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '02h23m11.5s', # From Xin+2021
               'DEC': '+42d59m30s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(0.0215)[0]), # redshift from Matthews et al 1964
               #'log10_dist': np.log10(93.6), # From London/Nikita Slides
               'log10_freq': np.log10(6.04e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 2: HS 1630+2355

In [6]:
log10_Mtot_to_Mc(9.86)

9.498764005203222

In [291]:
# informative mass prior
source_name = 'HS_1630+2355'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h33m02.7s', # From Xin+2021
               'DEC': '+23d49m28.8s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(0.82101)[0]), # redshift from SIMBAD
               #'log10_dist': np.log10(5268), # From Caitlin
               'log10_freq': np.log10(11.3e-9), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.86), # From Xin+2021
               #'log10_Mc': 9.86, # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [292]:
# uninformative mass prior (log-uniform)
source_name = 'HS_1630+2355_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h33m02.664s', # From Caitlin
               'DEC': '+23d49m28.52s', # From Caitlin
               'log10_dist': np.log10(z_to_dist_L(0.82101)[0]), # redshift from SIMBAD/Graham+2015 redshift
               #'log10_dist': np.log10(5268), # From Caitlin
               'log10_freq': np.log10(11.3e-9), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [293]:
# informative mass prior + bogus sky location
source_name = 'HS_1630+2355_altskyloc'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '20h00m00.000s', # bogus sky location
               'DEC': '+00d00m00.00s', # bogus sky location
               'log10_dist': np.log10(z_to_dist_L(0.82101)[0]), # redshift from SIMBAD/Graham+2015 redshift
               #'log10_dist': np.log10(5268), # From Caitlin
               'log10_freq': np.log10(11.3e-9), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.86), # From Xin+2021
               #'log10_Mc': 9.86, # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [ ]:
'/home/bbl29/targeted_cws_ng15/priors/PKS_J0851-0111_priors.json'

In [351]:
# uninformative mass prior + bogus sky location
source_name = 'HS_1630+2355_altskyloc_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '20h00m00.000s', # bogus sky location
               'DEC': '+00d00m00.00s', # bogus sky location
               'log10_dist': np.log10(z_to_dist_L(0.82101)[0]), # redshift from SIMBAD/Graham+2015 redshift
               #'log10_dist': np.log10(5268), # From Caitlin
               'log10_freq': np.log10(11.3e-9), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### older run

In [294]:
# old run - using arbitrary sigma error
source_name = 'HS1630'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h33m02.664s', # From Caitlin
               'DEC': '+23d49m28.52s', # From Caitlin
               'log10_dist': np.log10(5268), # From Caitlin
               'log10_freq': np.log10(11.3e-9), # From Xin+2021
               'log10_Mc': 9.86, # From Xin+2021
               'log10_Mc_sigma': 0.5}) # Arbitrary

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 3: SDSS J164452.71+4307

In [295]:
# informative mass priors
source_name = 'SDSS_J164452.71+4307'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h44m52.7s', # From Xin+2021
               'DEC': '+43d07m52.9s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(1.712983)[0]), # redshift from SDSS
               #'log10_dist': np.log10(13090), # From London/Nikita Slides
               'log10_freq': np.log10(1.16e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(10.15), # From Xin+2021
               #'log10_Mc': 10.15), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [296]:
# uninformative mass priors
source_name = 'SDSS_J164452.71+4307_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h44m52.7s', # From Xin+2021
               'DEC': '+43d07m52.9s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(1.712983)[0]), # redshift from SDSS
               #'log10_dist': np.log10(13090), # From London/Nikita Slides
               'log10_freq': np.log10(1.16e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 4: SDSS J114857.33+1600

In [337]:
np.log10(z_to_dist_L(1.224)[0])

3.936377274835147

In [338]:
# informative mass priors
source_name = 'SDSS_J114857.33+1600'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '11h48m57.4s', # From Xin+2021
               'DEC': '+16d00m22.7s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(1.224)[0]), # redshift from Graham+2015
               #'log10_dist': np.log10(8610), # From London/Nikita Slides
               'log10_freq': np.log10(1.25e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.9), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [339]:
# uninformative mass priors
source_name = 'SDSS_J114857.33+1600_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '11h48m57.4s', # From Xin+2021
               'DEC': '+16d00m22.7s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(1.224)[0]), # redshift from Graham+2015
               #'log10_dist': np.log10(8610), # From London/Nikita Slides
               'log10_freq': np.log10(1.25e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 5: HS 0926+3608

In [340]:
np.log10(z_to_dist_L(2.150)[0])

4.238681637416932

In [299]:
# informative mass priors
source_name = 'HS_0926+3608'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '09h29m52.1s', # From Xin+2021
               'DEC': '+35d54m49.6s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(2.150)[0]), # redshift from Graham+2015
               #'log10_dist': np.log10(17290), # From London/Nikita Slides
               'log10_freq': np.log10(1.48e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.95), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [300]:
# uninformative mass priors
source_name = 'HS_0926+3608_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '09h29m52.1s', # From Xin+2021
               'DEC': '+35d54m49.6s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(2.150)[0]), # redshift from Graham+2015
               #'log10_dist': np.log10(17290), # From London/Nikita Slides
               'log10_freq': np.log10(1.48e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 4: SDSS J092911.35+2037

In [301]:
# informative mass priors
source_name = 'SDSS_J092911.35+2037'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '09h29m11.3s', # From Xin+2021
               'DEC': '+20d37m09.2s', # From Xin+2021
               'log10_dist': np.log10(dist_L(1.8491)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.30e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.92), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [302]:
# uninformative mass priors
source_name = 'SDSS_J092911.35+2037_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '09h29m11.3s', # From Xin+2021
               'DEC': '+20d37m09.2s', # From Xin+2021
               'log10_dist': np.log10(dist_L(1.8491)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.30e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 5: SDSS J133516.17+1833

In [303]:
# informative mass priors
source_name = 'SDSS_J133516.17+1833'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h35m16.1s', # From Xin+2021
               'DEC': '+18d33m41.8s', # From Xin+2021
               'log10_dist': np.log10(dist_L(1.19170)[0]), # calculated from SIMBAD/Graham+2015 redshift
               #'log10_dist': np.log10(8335), # From London/Nikita Slides
               'log10_freq': np.log10(1.34e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.76), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [304]:
# uninformative mass priors
source_name = 'SDSS_J133516.17+1833_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h35m16.1s', # From Xin+2021
               'DEC': '+18d33m41.8s', # From Xin+2021
               'log10_dist': np.log10(dist_L(1.19170)[0]), # calculated from SIMBAD/Graham+2015 redshift
               #'log10_dist': np.log10(8335), # From London/Nikita Slides
               'log10_freq': np.log10(1.34e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 6: SDSS J140704.43+2735

In [305]:
# informative mass priors
source_name = 'SDSS_J140704.43+2735'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '14h07m04.5s', # From Xin+2021
               'DEC': '+27d35m56.3s', # From Xin+2021
               'log10_dist': np.log10(dist_L(2.2162)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.48e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.94), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [306]:
# uninformative mass priors
source_name = 'SDSS_J140704.43+2735_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '14h07m04.5s', # From Xin+2021
               'DEC': '+27d35m56.3s', # From Xin+2021
               'log10_dist': np.log10(dist_L(2.2162)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.48e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 7: SDSS J134855.27-0321

In [307]:
# informative mass priors
source_name = 'SDSS_J134855.27-0321'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h48m55.3s', # From Xin+2021
               'DEC': '-3d21m41.4s', # From Xin+2021
               'log10_dist': np.log10(dist_L(2.0986)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.62e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.89), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [308]:
# uninformative mass priors
source_name = 'SDSS_J134855.27-0321_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h48m55.3s', # From Xin+2021
               'DEC': '-3d21m41.4s', # From Xin+2021
               'log10_dist': np.log10(dist_L(2.0986)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.62e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 10: SDSS J160730.33+1449

In [309]:
# informative mass priors
source_name = 'SDSS_J160730.33+1449'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h07m30.3s', # From Xin+2021
               'DEC': '+14d49m04.2s', # From Xin+2021
               'log10_dist': np.log10(dist_L(1.8013)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.34e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.82), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [310]:
# uninformative mass priors
source_name = 'SDSS_J160730.33+1449_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '16h07m30.3s', # From Xin+2021
               'DEC': '+14d49m04.2s', # From Xin+2021
               'log10_dist': np.log10(dist_L(1.8013)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.34e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 11: SDSS J131706.19+2714

In [311]:
# informative mass priors
source_name = 'SDSS_J131706.19+2714'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h17m06.2s', # From Xin+2021
               'DEC': '+27d14m16.7s', # From Xin+2021
               'log10_dist': np.log10(dist_L(2.6681)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.39e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.35), # From Xin+2021 table 3
               'log10_Mc_sigma': 0.6}) # Doubled to account for disagreement with table 1

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [312]:
# uninformative mass priors
source_name = 'SDSS_J131706.19+2714_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h17m06.2s', # From Xin+2021
               'DEC': '+27d14m16.7s', # From Xin+2021
               'log10_dist': np.log10(dist_L(2.6681)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.39e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 12: SNU J13120+0641

In [313]:
# informative mass priors
source_name = 'SNU_J13120+0641'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h12m04.7s', # From Xin+2021
               'DEC': '+06d41m07.6s', # From Xin+2021
               'log10_dist': np.log10(dist_L(0.24182)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.55e-08), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(9.14), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [314]:
# uninformative mass priors
source_name = 'SNU_J13120+0641_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '13h12m04.7s', # From Xin+2021
               'DEC': '+06d41m07.6s', # From Xin+2021
               'log10_dist': np.log10(dist_L(0.24182)[0]), # calculated from SDSS redshift
               'log10_freq': np.log10(1.55e-08), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 13: OJ287

- Use redshift: 0.306 from SIMBAD
- Use q = 0.008 for chirp mass

In [341]:
np.log10(z_to_dist_L(0.306)[0])

3.2063584825007037

In [353]:
log10_Mtot_to_Mc(10.26, q=0.008), log10_Mtot_to_Mc(10.26, q=0.08)

(8.997701353663757, 9.561745485610826)

In [354]:
# informative mass priors
source_name = 'OJ287'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '08h54m48.9s', # From Xin+2021
               'DEC': '+20d06m31s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(0.306)[0]), # calculated from SIMBAD redshift
               'log10_freq': np.log10(5.82e-09), # From Xin+2021
               'log10_Mc': log10_Mtot_to_Mc(10.26, q=0.008), # From Xin+2021
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [355]:
# uninformative mass priors
source_name = 'OJ287_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '08h54m48.9s', # From Xin+2021
               'DEC': '+20d06m31s', # From Xin+2021
               'log10_dist': np.log10(z_to_dist_L(0.306)[0]), # calculated from SIMBAD redshift
               'log10_freq': np.log10(5.82e-09), # From Xin+2021
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 14: PKS 2131-021

In [317]:
# informative mass priors
source_name = 'PKS_2131-021'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '21h34m10.309s', # From SIMBAD
               'DEC': '-01d53m17.238s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(1.285)[0]), # redshift from O'Neill et al 2022
               #'log10_dist': np.log10(9080), # from O'Neill et al 2022
               'log10_freq': np.log10(1.3e-8), # reported fGW from O'Neill et al 2022
               'log10_Mc': 9.3, # selected such that upper limit from O'Neill et al 2022 placed at 1-sigma
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [318]:
# uninformative mass priors
source_name = 'PKS_2131-021_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '21h34m10.309s', # From SIMBAD
               'DEC': '-01d53m17.238s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(1.285)[0]), # redshift from O'Neill et al 2022
               #'log10_dist': np.log10(9080), # from O'Neill et al 2022
               'log10_freq': np.log10(1.3e-8), # from O'Neill et al 2022
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 15: NGC 3115

In [7]:
# run at 16 nHz, informative mass priors
source_name = 'NGC_3115_16nHz'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '10h05m13.927s', # From SIMBAD
               'DEC': '-07d43m06.983s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), # redshift from SIMBAD
               #'log10_dist': np.log10(9.7), # From London/Nikita Slides
               'log10_freq': np.log10(16e-9), # 16 nHz
               'log10_Mc': log10_Mtot_to_Mc(np.log10(2e9)), # Wong+2011
               'log10_Mc_sigma': 0.5}) # Arbitrary

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [320]:
# run at 16 nHz, uninformative mass priors
source_name = 'NGC_3115_16nHz_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '10h05m13.927s', # From SIMBAD
               'DEC': '-07d43m06.983s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), # redshift from SIMBAD
               #'log10_dist': np.log10(9.7), # From London/Nikita Slides
               'log10_freq': np.log10(16e-9), # 16 nHz
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [11]:
# run at 48.9 nHz, informative mass priors
source_name = 'NGC_3115_49nHz'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '10h05m13.927s', # From SIMBAD
               'DEC': '-07d43m06.983s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), # redshift from SIMBAD
               #'log10_dist': np.log10(9.7), # From London/Nikita Slides
               'log10_freq': -7.31, # 49 nHz
               'log10_Mc': log10_Mtot_to_Mc(np.log10(2e9)), # Wong+2011
               'log10_Mc_sigma': 0.5}) # Arbitrary

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [5]:
# run at 48.9 nHz, uninformative mass priors
source_name = 'NGC_3115_49nHz_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '10h05m13.927s', # From SIMBAD
               'DEC': '-07d43m06.983s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), # redshift from SIMBAD
               #'log10_dist': np.log10(9.7), # From London/Nikita Slides
               'log10_freq': -7.31, # 49 nHz
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [321]:
# initial arbitrary frequency run
source_name = 'NGC_3115'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '10h05m13.927s', # From SIMBAD
               'DEC': '-07d43m06.983s', # From SIMBAD
               'log10_dist': np.log10(z_to_dist_L(0.002222)[0]), # redshift from SIMBAD
               #'log10_dist': np.log10(9.7), # From London/Nikita Slides
               'log10_freq': -7.32, # We don't have a prior frequency, using this from initial trial
               'log10_Mc': log10_Mtot_to_Mc(2e9), # Wong+2011
               'log10_Mc_sigma': 0.5}) # Arbitrary

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 16: PKS J0805-0111

Note in de la Parra, they estimate 10 Gpc, which is consistent with the redshift $z = 1.388$

In [349]:
# informative mass priors
source_name = 'PKS_J0805-0111'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '08h05m12.888s', # From SIMBAD
               'DEC': '-01d11m13.795s', # From SIMBA
               'log10_dist': np.log10(z_to_dist_L(1.388)[0]), # redshift from SIMBAD, confirmed de la Parra 2024
               'log10_freq': np.log10(18.7e-9), # reported fGW from de la Parra 2024
               'log10_Mc': 9.4, # selected such that upper limit from de la Parra 2024 placed at 1-sigma
               'log10_Mc_sigma': 0.3}) # Standard

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [350]:
# uninformative mass priors
source_name = 'PKS_J0805-0111_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': '08h05m12.888s', # From SIMBAD
               'DEC': '-01d11m13.795s', # From SIMBA
               'log10_dist': np.log10(z_to_dist_L(1.388)[0]), # redshift from SIMBAD, confirmed de la Parra 2024
               'log10_freq': np.log10(18.7e-9), # reported fGW from de la Parra 2024
               'log10_Mc': None,
               'log10_Mc_sigma': None})

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 17: MCG 5-40-026

This is a target from Forrest I am following up on using parallel tempering. We'd like to do a varied frequency run with a log-uniform prior from 1/Tspan to 100 nHz.

In [33]:
# uninformative mass priors
source_name = 'MCG_5-40-026_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': "17h01m07.80s", # From Forrest/Rohan
               'DEC': "+29d24m24.6s", # From Forrest/Rohan
               'log10_dist': 2.18723862, # from Forrest/Rohan
               'log10_freq': -7.784086401336019, # from Forrest/Rohan
               'log10_Mc': None, # none for now
               'log10_Mc_sigma': None}) # none for now

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

source_name = 'MCG_5-40-026'
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

### 18: CSO J1735+5049

This is a target suggested by Mitch Begelman. The observed orbital period is 2417 days

In [32]:
# uninformative mass priors
source_name = 'CSO_J1735+5049_UL'
priors = copy.deepcopy(prior_template)
priors.update({'RA': "17h35m49.01s", # From Kiehlmann+2023
               'DEC': "+50d49m11.57s", # From Kiehlmann+2023
               'log10_dist': np.log10(z_to_dist_L(0.835)[0]), # redshift from Kiehlmann+2023
               'log10_freq': np.log10(2/(2417*u.day.to(u.s))), # orbital period in days from chiara
               'log10_Mc': None, # none for now
               'log10_Mc_sigma': None}) # none for now

# save kwargs
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))
    
source_name = 'CSO_J1735+5049'
fname = f'{project_path}/priors/{source_name}_priors.json'
with open(fname, 'w') as fout:
    json.dump(priors, fout, sort_keys=False, indent=4,
              separators=(',', ': '))

In [35]:
44/16

2.75